# Loading a Pretrained Model

In [1]:
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torch.hub.load("rasbt/ord-torchhub", model="resnet34_coral_afad", source='github', pretrained=True)
model.to(device);

Using cache found in /Users/sebastianraschka/.cache/torch/hub/rasbt_ord-torchhub_main
Downloading: "https://github.com/rasbt/ord-torchhub/releases/download/1.0.0/resnet34_coral_afad.pt" to /Users/sebastianraschka/.cache/torch/hub/checkpoints/resnet34_coral_afad.pt
28.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


# Classifying Images (from AFAD)

Note that the model has been pretrained on the [AFAD dataset](https://afad-dataset.github.io) and may not perform well on other datasets.

In [2]:
from PIL import Image
from torchvision import transforms

img1 = Image.open("example1.jpg")
img2 = Image.open("example2.jpg")

resize = transforms.Compose([transforms.Resize((120, 120)),
                             transforms.ToTensor()])
img1 = resize(img1)
img2 = resize(img2)
batch = torch.stack((img1, img2))

In [3]:
with torch.no_grad():
    logits = model(batch)

In [4]:
def label_from_logits(logits):
    """ Converts logits to class labels.
    This is function is specific to CORAL.
    """
    probas = torch.sigmoid(logits)
    predict_levels = probas > 0.5
    predicted_labels = torch.sum(predict_levels, dim=1)
    return predicted_labels

In [5]:
predictions = label_from_logits(logits)

print(f'Rank indices [0-13]: {predictions}')
print(f'Real ages [18-30]: {predictions+18}')

Rank indices [0-13]: tensor([7, 6])
Real ages [18-30]: tensor([25, 24])


# 

# Transfer Learning

Use the pretrained model and adjust it to the number of classes in your dataset:

```python
model = torch.hub.load("rasbt/ord-torchhub", model="resnet34_coral_afad", source='github', pretrained=True)

NUM_CLASSES = # number of classes in your dataset
model.output_layer = torch.nn.Linear(512, 1, bias=False)
model.output_biases = torch.nn.Parameter(
    torch.zeros(NUM_CLASSES-1).float())
```

Then, take a look at the [../_train/resnet34_coral_afad.py](../_train/resnet34_coral_afad.py) file for defining the training loop.

Essentially, you need to replace the 


    ##########################
    # MODEL
    ##########################


section with the model above. And adjust the 

    ############################
    # Dataset
    ############################
    
section based on your dataset.